Установка зависимостей

In [ ]:
!pip3.9 install -r ./requirements.txt --extra-index-url https://download.pytorch.org/whl/cu116

Прослушивание новых после генерации голосов

In [2]:
from ss14tts import model, speakers
from IPython.display import Audio, display
from src.SpeakerPatch import dynVoices
import random, glob, os

sample_rate = 24000
#example_text = ["Съешь же ещё этих мягких французских булок, да выпей чаю.",
#        "Клоун, прекрати разбрасывать банановые кожурки офицерам под ноги!",
#        "Капитан, вы уверены что хотите назначить клоуна на должность главы персонала?",
#        "Эс Бэ! Тут человек в сером костюме, с тулбоксом и в маске! Помогите!!"]
example_text = ["Hi. My name is amogus"]
for voiceFile in glob.glob(dynVoices+'/tmp/*.pt'):
    audio = model.apply_tts(ssml_text='<speak><prosody rate="fast">'+random.choice(example_text)+'</prosody></speak>', speaker="random",voice_path=voiceFile,sample_rate=sample_rate)
    speaker = os.path.relpath(voiceFile,dynVoices)
    print('speaker {}'.format(speaker))
    display(Audio(audio, rate=sample_rate))


Loaded voice from E:\RiderProjects\ss14-tts-api/voices/tmp\Voice1.pt
speaker tmp\Voice1.pt


Loaded voice from E:\RiderProjects\ss14-tts-api/voices/tmp\Voice10.pt
speaker tmp\Voice10.pt


Loaded voice from E:\RiderProjects\ss14-tts-api/voices/tmp\Voice11.pt
speaker tmp\Voice11.pt


Loaded voice from E:\RiderProjects\ss14-tts-api/voices/tmp\Voice12.pt
speaker tmp\Voice12.pt


Loaded voice from E:\RiderProjects\ss14-tts-api/voices/tmp\Voice13.pt
speaker tmp\Voice13.pt


...

Loaded voice from E:\RiderProjects\ss14-tts-api/voices/tmp\Voice5.pt
speaker tmp\Voice5.pt


Loaded voice from E:\RiderProjects\ss14-tts-api/voices/tmp\Voice6.pt
speaker tmp\Voice6.pt


Loaded voice from E:\RiderProjects\ss14-tts-api/voices/tmp\Voice7.pt
speaker tmp\Voice7.pt


Loaded voice from E:\RiderProjects\ss14-tts-api/voices/tmp\Voice8.pt
speaker tmp\Voice8.pt


Loaded voice from E:\RiderProjects\ss14-tts-api/voices/tmp\Voice9.pt
speaker tmp\Voice9.pt


Генератор голосов

In [1]:
from ss14tts import model
from src.AutoGenerator import Generate

for voiceFile in Generate(model,20):  # 20 голосов создает, голоса будет идти с 1
    print(voiceFile)


Start download silero models


100.0%


Generated new voice


ValueError: 

Тестирование голосов

In [ ]:
from ss14tts import model, speakers
from IPython.display import Audio, display
from src.SpeakerPatch import dynVoices
import random, glob, os

sample_rate = 24000
example_text = ["Съешь же ещё этих мягких французских булок, да выпей чаю.",
        "Клоун, прекрати разбрасывать банановые кожурки офицерам под ноги!",
        "Капитан, вы уверены что хотите назначить клоуна на должность главы персонала?",
        "Эс Бэ! Тут человек в сером костюме, с тулбоксом и в маске! Помогите!!"]

for voiceFile in glob.glob(dynVoices+'/*.pt'):
    speaker = os.path.relpath(voiceFile,dynVoices)
    print('speaker {} with {}'.format(speaker, voiceFile))
    for example in example_text:
        audio = model.apply_tts(ssml_text='<speak><prosody rate="fast">'+example+'</prosody></speak>', speaker="random",voice_path=voiceFile,sample_rate=sample_rate)
        display(Audio(audio, rate=sample_rate))
